- 자연어 처리(NLP)  : 인간의 언어를 컴퓨터가 이해하고 처리 할수 있도록 하는 인공지능
- 주요 요소
  - 토큰화(Tokenization) : 문장을 단어 또는 문장단위로 나누는 과정
  - 품사 태깅(POS Tagging) : 각 단어의 품사(명사,동사)를 분석
  - 어휘 및 의미 분석(Semantic Analysis) : 단어간의 관계를 분석 문맥을 파악
  - 구분 분석(Syntactic Parsing): 문장의 문법적 구조를분석
  - 감성 분석(Sentiment Analysis) : 텍스트에서 감정을 분석(긍정, 부정)

- NLP 활용
  - 챗봇
    - 고객 서비스 자동화
    - 의료 상담 보조
    - 금융 상담 보조
  - 기계 번역
    - 구글번역
    - 네이버 파파고
    - 딥러닝 기반 번역 서비스
  - 텍스트 분석
    - 감성 분석
    - 자동 요약(뉴스요약)
    - 키워드 추출 및 정보 검색

In [1]:
!pip install transformers

감성분석

In [10]:
from transformers import pipeline
# 감성 분석 모델 로드
classifier = pipeline("sentiment-analysis")
# 예제 텍스트 분석
text = ["I love this movie!","This product is terrible..."]
result = classifier(text)

for t, r in zip(text,result):
  print(f"Text : {t}=> sentiment: {r['label']}  :  {r['score']:.2f}")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Text : I love this movie!=> sentiment: POSITIVE  :  1.00
Text : This product is terrible...=> sentiment: NEGATIVE  :  1.00


기계번역

In [13]:
from transformers import pipeline
# 번역모델 로드
translator = pipeline(model = "Helsinki-NLP/opus-mt-ko-en")
# 번역할 문장
text = "안녕하세요, 반갑습니다."
# 번역 실행
translation = translator(text)
print(f"original : {text}\ntranslated : {translation}")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


original : 안녕하세요, 반갑습니다.
translated : [{'translation_text': 'Hello. Nice to meet you.'}]


질문 답변 시스템

In [14]:
from transformers import pipeline
# 질문 답변 모델 로드
qa_pipeline = pipeline("question-answering")
# 문맥과 질문 입력
context = "Albert Einstein was a theoretical physicist who developed the theory of relativity."
quest = "Who developed the theory of relativity?"
# 답변 생성
answer = qa_pipeline(question=quest, context=context)
# 결과 출력
print(f"Question: {quest}\nAnswer: {answer['answer']}")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


Question: Who developed the theory of relativity?
Answer: Albert Einstein
